In [1]:
import pandas as pd
df = pd.read_csv('./data/movies_train.csv')
testdf = pd.read_csv('./data/movies_test.csv')
# 지수 표현 제거
pd.options.display.float_format = '{:.5f}'.format

In [163]:
df.corr()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
time,1.00000,0.26468,0.30673,0.62321,0.11415,0.44145
dir_prev_bfnum,0.26468,1.00000,0.13182,0.32352,0.08382,0.28318
dir_prev_num,0.30673,0.13182,1.00000,0.45071,0.01401,0.25967
num_staff,0.62321,0.32352,0.45071,1.00000,0.07787,0.54426
num_actor,0.11415,0.08382,0.01401,0.07787,1.00000,0.11118
box_off_num,0.44145,0.28318,0.25967,0.54426,0.11118,1.00000


# 전처리

In [2]:
# column 생성
def make_col(dataframe, column_name, mapper) :
    new = []
    for i in dataframe[column_name] :
        if i in mapper.keys() :
            new.append(mapper.get(i))
        else :
            new.append(1)
    dataframe['new_' + str(column_name)] = pd.DataFrame(new)
    del dataframe[column_name]
    return dataframe

## title
title 은 제거

In [3]:
del df['title']
del testdf['title']

## distributor

In [4]:
a = df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False)

distributor_mapper = {}
for i in range(len(a)) :
    if a[i] >= 1000000 :
        distributor_mapper.update({a.index[i] : 5 })
    elif a[i] >= 100000 :
        distributor_mapper.update({a.index[i] : 4 })
    elif a[i] >= 10000 :
        distributor_mapper.update({a.index[i] : 3 })
    elif a[i] >= 1000 :
        distributor_mapper.update({a.index[i] : 2 })
    else :
        distributor_mapper.update({a.index[i] : 1 })

df = make_col(df, 'distributor', distributor_mapper )
testdf = make_col(testdf, 'distributor', distributor_mapper )

### 배급사별 box_off_num 의 평균에 따른 head, tail

In [18]:
df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False).head(20)

distributor
CJ E&M Pictures    4.122337e+06
(주)쇼박스             3.386656e+06
(주)아이필름코퍼레이션       3.117859e+06
쇼박스(주)미디어플렉스       2.634823e+06
영구아트무비             2.541603e+06
CJ E&M 영화부문        2.505309e+06
(주)NEW             2.184405e+06
CJ 엔터테인먼트          2.162742e+06
이십세기폭스코리아(주)       1.637046e+06
SK텔레콤(주)           1.383867e+06
CGV아트하우스           1.318189e+06
롯데엔터테인먼트           1.238149e+06
전망좋은영화사            1.214237e+06
주머니필름              1.201944e+06
NEW                1.019017e+06
싸이더스               9.634390e+05
시네마서비스             8.968432e+05
보람엔터테인먼트           8.496350e+05
스튜디오 블루            7.716990e+05
씨네그루(주)키다리이엔티      7.551840e+05
Name: box_off_num, dtype: float64

In [19]:
df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False).tail(20)

distributor
웃기씨네           464.5
판다미디어          432.0
(주)더픽쳐스        429.0
이상우필름          339.0
(주)온비즈넷        310.0
(주)유비네트워크      303.0
(주)휘엔터테인먼트     228.0
크리에이티브컴즈(주)    188.0
ysfilm         172.0
에스와이코마드        163.0
건시네마           162.0
(주)씨엠닉스        136.0
위더스필름           90.0
인터콘미디어          79.0
한국YWCA연합회       71.0
나우콘텐츠           54.0
위드시네마           46.0
사람과 사람들         42.0
고구마공작소           8.0
인피니티엔터테인먼트       2.0
Name: box_off_num, dtype: float64

## genre

### genre 에 따른 box_off_num 평균 확인

In [83]:
df.groupby(['genre']).mean()['box_off_num'].sort_values(ascending=False).index

Index(['느와르', '액션', 'SF', '코미디', '드라마', '공포', '미스터리', '멜로/로맨스', '애니메이션',
       '서스펜스', '다큐멘터리', '뮤지컬'],
      dtype='object', name='genre')

In [84]:
df.groupby(['genre']).mean()['box_off_num'].sort_values(ascending=False)

genre
느와르       2.263695e+06
액션        2.203974e+06
SF        1.788346e+06
코미디       1.193914e+06
드라마       6.256898e+05
공포        5.908325e+05
미스터리      5.275482e+05
멜로/로맨스    4.259680e+05
애니메이션     1.819267e+05
서스펜스      8.261100e+04
다큐멘터리     6.717226e+04
뮤지컬       6.627000e+03
Name: box_off_num, dtype: float64

In [185]:
df['genre'].value_counts()

드라마       221
다큐멘터리      93
멜로/로맨스     78
코미디        53
공포         42
액션         28
느와르        27
애니메이션      21
미스터리       17
SF         13
뮤지컬         5
서스펜스        2
Name: genre, dtype: int64

### genre_version1

In [168]:
df = pd.get_dummies(df, columns=['genre'], prefix='genre')
testdf = pd.get_dummies(testdf, columns=['genre'], prefix='genre')

### genre_version2

version1, 3 보다 좋음

영화 종류 묶기 + box_off_num의 평균에 따라 비선호 영화 묶기

=> 1466134.4524178207	

* 드라마 멜로/로맨스6

* 느와르 액션5

* sf4

* 코미디3

* 공포 미스터리 서스펜스2

* 애니메이션 다큐멘터리 뮤지컬1

In [5]:
genre_mapper = {'느와르' : 5 , '액션' : 5, 'SF' : 4, '코미디' : 3, '드라마' : 6, '공포' : 2, '미스터리' : 2, '멜로/로맨스' : 6, '애니메이션' : 1,
       '서스펜스' : 2, '다큐멘터리' : 1, '뮤지컬' : 1} 

df = make_col(df, 'genre', genre_mapper )
testdf = make_col(testdf, 'genre', genre_mapper )

### genre_version3

box_off_num 평균에 따라 임의로 분류

=> 1467553.925063711 

In [ ]:
느와르       4
액션        4
SF        3
코미디       3
드라마       2
공포        2
미스터리      2
멜로/로맨스    2
애니메이션     1
서스펜스     1
다큐멘터리     1
뮤지컬       1

In [210]:
genre_mapper = {'느와르' : 4 , '액션' : 4, 'SF' : 3, '코미디' : 3, '드라마' : 2, '공포' : 2, '미스터리' : 2, '멜로/로맨스' : 2, '애니메이션' : 1,
       '서스펜스' : 1, '다큐멘터리' : 1, '뮤지컬' : 1} 

df = make_col(df, 'genre', genre_mapper )
testdf = make_col(testdf, 'genre', genre_mapper )

## release_time

* days 그대로 사용해보자

=> sklearn,  1484884.4136596955

=> 전처리 하는 것이 더 좋음


In [6]:
def release_time(df) :
    years, months, days = [], [], []
    for line in df['release_time'] :
        a = line.split('-')
        years.append(int(a[0]))
        months.append(int(a[1]))
        days.append(int(a[2]))
        
    new_days = []
    for i in range(len(days)) :
        if int(days[i]) >= 20 :
            new_days.append(3)
        elif int(days[i]) >= 10 :
            new_days.append(2)
        else :
            new_days.append(1)    
            
    df['release_year'] = pd.DataFrame(years)
    df['release_month'] = pd.DataFrame(months)
    df['release_day'] = pd.DataFrame(new_days)

    del df['release_time']
    return df

df = release_time(df)
testdf = release_time(testdf)

## time

전처리 이전과 비교하였을 때, box_off_num 과의 상관관계는 낮아졌다 ( 0.44 -> 0.29 ) 

하지만, 전처리 이후 성능이 개선됨

=> 1472434.761373017 ( 130, 70, ~ )

=> 1472973.0197688688 ( 130, 100, 70, ~ )

In [7]:
def time(df) :
    new_time = []
    for i in df['time'] :
        if i >= 130 :
            new_time.append(3)
        elif i > 70 : 
            new_time.append(2)
        else :
            new_time.append(1)

    df['new_time'] = pd.DataFrame(new_time)
    del df['time']
    return df

df = time(df)
testdf = time(testdf)

## screening_rat

### mapping_version1

version1 이  version2보다 더 좋음

In [88]:
screening_rat_mapper = {'15세 관람가' : 3, '12세 관람가' : 2, '전체 관람가' : 2, '청소년 관람불가' : 1} 

df = make_col(df, 'screening_rat', screening_rat_mapper )
testdf = make_col(testdf, 'screening_rat', screening_rat_mapper )

### mapping_version2

In [83]:
def screen_rat(df) :
    tmp = sorted(df['screening_rat'].unique(), reverse=False)

    mapping = {}
    num = 1
    for i in tmp :
        mapping.update({i : num})
        num += 1

    return make_col(df, 'screening_rat', mapping)

In [84]:
df = screen_rat(df)
testdf = screen_rat(testdf)

### mapping_version3

version1과 큰 차이는 나지 않지만 더 좋음

=> 1483501.7131900766	

In [8]:
df = pd.get_dummies(df, columns=['screening_rat'], prefix='screening_rat')
testdf = pd.get_dummies(testdf, columns=['screening_rat'], prefix='screening_rat')

### 관람 연령 제한에 따른 box_off_num 평균, 영화 수 

In [51]:
df.groupby(['screening_rat']).mean()['box_off_num'].sort_values(ascending=False)

screening_rat
15세 관람가     1.247519e+06
12세 관람가     8.449809e+05
청소년 관람불가    3.641813e+05
전체 관람가      1.351005e+05
Name: box_off_num, dtype: float64

In [52]:
df.groupby(['screening_rat']).count()['box_off_num'].sort_values(ascending=False)

screening_rat
청소년 관람불가    204
15세 관람가     202
12세 관람가     102
전체 관람가       92
Name: box_off_num, dtype: int64

## director

In [9]:
del df['director']
del testdf['director']

## dir_prev_bfnum

### version1
* 결측치 0으로 채워 넣음

In [10]:
df = df.fillna(0)
testdf = testdf.fillna(0)

### version2
* 결측치가 많은 column이므로 제거

=> 1483572.7123881471	

In [10]:
del df['dir_prev_bfnum']
del testdf['dir_prev_bfnum']

## dir_prev_num

* column제거하였더니 점수 떨어짐

=> 1471931.2287102737	

In [11]:
del df['dir_prev_num']
del testdf['dir_prev_num']

## num_staff

staff 는 범주화하지 않는 것이 더 좋다!

=>	1536288.9882051074

### staff_version1

In [68]:
def staff(dataframe) : 
    new_staff = []
    for i in dataframe['num_staff'] :
        if i == 0 :
            new_staff.append(0)
        elif i <= 100 :
            new_staff.append(1)
        elif i <= 300 :
            new_staff.append(2)        
        elif i <= 500 :
            new_staff.append(3)
        else :
            new_staff.append(4)
    dataframe['new_staff'] = pd.DataFrame(new_staff)
    del dataframe['num_staff']
    
    return dataframe

In [69]:
df = staff(df)
testdf = staff(testdf)

In [70]:
df.groupby(['new_staff']).mean()['box_off_num'].sort_values(ascending=False)

new_staff
4    4.132596e+06
3    1.980997e+06
2    6.495566e+05
1    1.164862e+05
0    7.805562e+03
Name: box_off_num, dtype: float64

## num_actor

column 제거했더니 ?

=> sklearn, 1470532.643567296	

In [12]:
del df['num_actor']
del testdf['num_actor']

In [17]:
df

,num_staff,box_off_num,new_distributor,new_genre,release_year,release_month,release_day,new_time,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,91,23398,5,5,2012,11,3,2,0,0,0,1
1,387,7072501,5,5,2015,11,2,3,0,0,0,1
2,343,6959083,5,5,2013,6,1,2,0,1,0,0
3,20,217866,5,3,2012,7,2,2,0,0,1,0
4,251,483387,5,3,2010,11,1,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
595,510,1475091,5,6,2014,8,2,2,0,0,0,1
596,286,1716438,5,6,2013,3,2,2,0,1,0,0
597,123,2475,3,2,2010,9,3,2,0,0,0,1
598,431,2192525,5,5,2015,5,2,2,0,1,0,0


# 상관관계

In [13]:
df.corr().sort_values(by='box_off_num', ascending = False)

,num_staff,box_off_num,new_distributor,new_genre,release_year,release_month,release_day,new_time,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
box_off_num,0.54426,1.00000,0.45016,0.09595,-0.00250,0.01910,-0.01102,0.29498,0.03390,0.21037,-0.13353,-0.13518
num_staff,1.00000,0.54426,0.69184,0.26775,-0.03289,-0.00284,0.00825,0.32914,0.06047,0.24653,-0.27374,-0.08569
new_distributor,0.69184,0.45016,1.00000,0.08755,-0.05986,-0.02799,-0.02770,0.25603,0.03496,0.25791,-0.16415,-0.16016
new_time,0.32914,0.29498,0.25603,0.10255,-0.05254,0.00813,0.02927,1.00000,-0.03082,0.04755,-0.07623,0.03498
screening_rat_15세 관람가,0.24653,0.21037,0.25791,0.06829,-0.07186,-0.05132,-0.01894,0.04755,-0.32242,1.00000,-0.30318,-0.51133
new_genre,0.26775,0.09595,0.08755,1.00000,-0.01883,-0.02042,0.00124,0.10255,0.01585,0.06829,-0.43499,0.25017
screening_rat_12세 관람가,0.06047,0.03390,0.03496,0.01585,-0.02142,0.04292,-0.02659,-0.03082,1.00000,-0.32242,-0.19260,-0.32483
release_month,-0.00284,0.01910,-0.02799,-0.02042,0.04663,1.00000,-0.05270,0.00813,0.04292,-0.05132,-0.00010,0.01724
release_year,-0.03289,-0.00250,-0.05986,-0.01883,1.00000,0.04663,0.02214,-0.05254,-0.02142,-0.07186,-0.06660,0.13932
release_day,0.00825,-0.01102,-0.02770,0.00124,0.02214,-0.05270,1.00000,0.02927,-0.02659,-0.01894,0.03884,0.01043


# 모델링

In [14]:
import numpy as np
from sklearn.metrics import make_scorer

def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)

    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    
    return score

rmse_score = make_scorer(rmse)
rmse_score

make_scorer(rmse)

## sklearn_Linear Regression

In [158]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [159]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
lm.fit(train_X, train_Y)

prediction = lm.predict(test_X)

## sklearn_RandomForestRegressor

=> 1528294.5021972428	 ( epoch X )

In [240]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=300,
                              n_jobs=-1,
                              random_state=46)
#score = cross_val_score(model, X_train, y_train, cv=20, scoring=rmse_score).mean()

In [242]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

model.fit(train_X, train_Y)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=-1, oob_score=False,
                      random_state=46, verbose=0, warm_start=False)

In [245]:
prediction = model.predict(test_X)

array([1.37224077e+06, 1.37712202e+06, 2.91496949e+06, 2.35731362e+06,
       2.18836183e+06, 3.83021800e+04, 3.05253000e+03, 1.25348333e+03,
       4.89507000e+03, 2.70432873e+05, 4.52084997e+05, 1.09368079e+06,
       2.43055558e+06, 3.80952000e+03, 1.54969315e+06, 2.49022232e+06,
       4.11237000e+03, 2.80051673e+06, 9.91018117e+05, 4.01313674e+06,
       3.31340000e+02, 2.75170133e+04, 1.04387323e+05, 2.15437233e+04,
       2.86300000e+02, 3.14427729e+06, 2.01759000e+03, 2.63354868e+06,
       9.63927000e+03, 7.76519000e+03, 4.01482357e+06, 2.53251757e+05,
       3.11146333e+03, 5.57792827e+05, 6.60871667e+03, 2.34236933e+04,
       1.94996333e+03, 4.38108000e+03, 4.30301000e+03, 3.68430000e+02,
       5.67607333e+03, 2.82403647e+05, 9.73643447e+05, 3.93951430e+05,
       1.05306354e+06, 1.88549011e+06, 8.36590000e+02, 1.48209400e+04,
       7.02168903e+05, 7.65833333e+02, 6.29552667e+03, 1.05540849e+06,
       5.76044333e+03, 1.29539582e+06, 1.30844980e+06, 2.60748967e+04,
      

## LGBM

=> 1480188.794477261

In [260]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=True, random_state = 777)

model = lgb.LGBMRegressor(random_state = 777, n_estimators=10000)
X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']


params = {
    'learning_rate' : [0.001, 0.0001, 0.00001],
    'min_child_samples' : [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid = params,
                 scoring=rmse_score,
                 cv=k_fold)

In [261]:
gs.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=777, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=10000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=777, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.0001, 1e-05],
              

In [262]:
gs.best_params_

{'learning_rate': 1e-05, 'min_child_samples': 30}

In [263]:
model = lgb.LGBMRegressor(random_state = 777, n_estimators=10000, learning_rate= 1e-05, min_child_samples= 30)
models = []
for train_idx, val_idx in k_fold.split(X_train) :
    x_t = X_train.iloc[train_idx]
    y_t = Y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = Y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set = (x_val, y_val), early_stopping_rounds=1000, verbose=100))
    

Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 4.39233e+12
[200]	valid_0's l2: 4.39124e+12
[300]	valid_0's l2: 4.39015e+12
[400]	valid_0's l2: 4.38906e+12
[500]	valid_0's l2: 4.38798e+12
[600]	valid_0's l2: 4.3869e+12
[700]	valid_0's l2: 4.38582e+12
[800]	valid_0's l2: 4.38474e+12
[900]	valid_0's l2: 4.38367e+12
[1000]	valid_0's l2: 4.3826e+12
[1100]	valid_0's l2: 4.38153e+12
[1200]	valid_0's l2: 4.38047e+12
[1300]	valid_0's l2: 4.37941e+12
[1400]	valid_0's l2: 4.37835e+12
[1500]	valid_0's l2: 4.37729e+12
[1600]	valid_0's l2: 4.37623e+12
[1700]	valid_0's l2: 4.37518e+12
[1800]	valid_0's l2: 4.37413e+12
[1900]	valid_0's l2: 4.37308e+12
[2000]	valid_0's l2: 4.37204e+12
[2100]	valid_0's l2: 4.37099e+12
[2200]	valid_0's l2: 4.36995e+12
[2300]	valid_0's l2: 4.36892e+12
[2400]	valid_0's l2: 4.36788e+12
[2500]	valid_0's l2: 4.36685e+12
[2600]	valid_0's l2: 4.36578e+12
[2700]	valid_0's l2: 4.36472e+12
[2800]	valid_0's l2: 4.36366e+12
[2900]	valid_0's l2: 4.3

[4100]	valid_0's l2: 5.47729e+12
[4200]	valid_0's l2: 5.47547e+12
[4300]	valid_0's l2: 5.47365e+12
[4400]	valid_0's l2: 5.47184e+12
[4500]	valid_0's l2: 5.47003e+12
[4600]	valid_0's l2: 5.46822e+12
[4700]	valid_0's l2: 5.46642e+12
[4800]	valid_0's l2: 5.46461e+12
[4900]	valid_0's l2: 5.46282e+12
[5000]	valid_0's l2: 5.46102e+12
[5100]	valid_0's l2: 5.45923e+12
[5200]	valid_0's l2: 5.45744e+12
[5300]	valid_0's l2: 5.45567e+12
[5400]	valid_0's l2: 5.4539e+12
[5500]	valid_0's l2: 5.45213e+12
[5600]	valid_0's l2: 5.45027e+12
[5700]	valid_0's l2: 5.44837e+12
[5800]	valid_0's l2: 5.44648e+12
[5900]	valid_0's l2: 5.44459e+12
[6000]	valid_0's l2: 5.44271e+12
[6100]	valid_0's l2: 5.44082e+12
[6200]	valid_0's l2: 5.43894e+12
[6300]	valid_0's l2: 5.43706e+12
[6400]	valid_0's l2: 5.43519e+12
[6500]	valid_0's l2: 5.43332e+12
[6600]	valid_0's l2: 5.43146e+12
[6700]	valid_0's l2: 5.4296e+12
[6800]	valid_0's l2: 5.42774e+12
[6900]	valid_0's l2: 5.42589e+12
[7000]	valid_0's l2: 5.42403e+12
[7100]	valid

[8200]	valid_0's l2: 3.15108e+12
[8300]	valid_0's l2: 3.14935e+12
[8400]	valid_0's l2: 3.14763e+12
[8500]	valid_0's l2: 3.1459e+12
[8600]	valid_0's l2: 3.14419e+12
[8700]	valid_0's l2: 3.14247e+12
[8800]	valid_0's l2: 3.14076e+12
[8900]	valid_0's l2: 3.13905e+12
[9000]	valid_0's l2: 3.13735e+12
[9100]	valid_0's l2: 3.13564e+12
[9200]	valid_0's l2: 3.13395e+12
[9300]	valid_0's l2: 3.13225e+12
[9400]	valid_0's l2: 3.13056e+12
[9500]	valid_0's l2: 3.12885e+12
[9600]	valid_0's l2: 3.12714e+12
[9700]	valid_0's l2: 3.12542e+12
[9800]	valid_0's l2: 3.12372e+12
[9900]	valid_0's l2: 3.12201e+12
[10000]	valid_0's l2: 3.12031e+12
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 3.12031e+12
Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 4.30428e+12
[200]	valid_0's l2: 4.30176e+12
[300]	valid_0's l2: 4.29926e+12
[400]	valid_0's l2: 4.29675e+12
[500]	valid_0's l2: 4.29425e+12
[600]	valid_0's l2: 4.29176e+12
[700]	valid_0's l2: 4.28927e+12
[800

[2000]	valid_0's l2: 1.93065e+12
[2100]	valid_0's l2: 1.92889e+12
[2200]	valid_0's l2: 1.92714e+12
[2300]	valid_0's l2: 1.92539e+12
[2400]	valid_0's l2: 1.92365e+12
[2500]	valid_0's l2: 1.92191e+12
[2600]	valid_0's l2: 1.92017e+12
[2700]	valid_0's l2: 1.91844e+12
[2800]	valid_0's l2: 1.91671e+12
[2900]	valid_0's l2: 1.91498e+12
[3000]	valid_0's l2: 1.91326e+12
[3100]	valid_0's l2: 1.91155e+12
[3200]	valid_0's l2: 1.90983e+12
[3300]	valid_0's l2: 1.90812e+12
[3400]	valid_0's l2: 1.90642e+12
[3500]	valid_0's l2: 1.90472e+12
[3600]	valid_0's l2: 1.90302e+12
[3700]	valid_0's l2: 1.90132e+12
[3800]	valid_0's l2: 1.89963e+12
[3900]	valid_0's l2: 1.89795e+12
[4000]	valid_0's l2: 1.89626e+12
[4100]	valid_0's l2: 1.89458e+12
[4200]	valid_0's l2: 1.89291e+12
[4300]	valid_0's l2: 1.89124e+12
[4400]	valid_0's l2: 1.8896e+12
[4500]	valid_0's l2: 1.88797e+12
[4600]	valid_0's l2: 1.88635e+12
[4700]	valid_0's l2: 1.88473e+12
[4800]	valid_0's l2: 1.88312e+12
[4900]	valid_0's l2: 1.88151e+12
[5000]	vali

[6400]	valid_0's l2: 3.56719e+12
[6500]	valid_0's l2: 3.5643e+12
[6600]	valid_0's l2: 3.5614e+12
[6700]	valid_0's l2: 3.55851e+12
[6800]	valid_0's l2: 3.55563e+12
[6900]	valid_0's l2: 3.55275e+12
[7000]	valid_0's l2: 3.54988e+12
[7100]	valid_0's l2: 3.54701e+12
[7200]	valid_0's l2: 3.54415e+12
[7300]	valid_0's l2: 3.5413e+12
[7400]	valid_0's l2: 3.53845e+12
[7500]	valid_0's l2: 3.5356e+12
[7600]	valid_0's l2: 3.53276e+12
[7700]	valid_0's l2: 3.52993e+12
[7800]	valid_0's l2: 3.5271e+12
[7900]	valid_0's l2: 3.52428e+12
[8000]	valid_0's l2: 3.52146e+12
[8100]	valid_0's l2: 3.51865e+12
[8200]	valid_0's l2: 3.51584e+12
[8300]	valid_0's l2: 3.51304e+12
[8400]	valid_0's l2: 3.51024e+12
[8500]	valid_0's l2: 3.50745e+12
[8600]	valid_0's l2: 3.50466e+12
[8700]	valid_0's l2: 3.50188e+12
[8800]	valid_0's l2: 3.49911e+12
[8900]	valid_0's l2: 3.49634e+12
[9000]	valid_0's l2: 3.49357e+12
[9100]	valid_0's l2: 3.49081e+12
[9200]	valid_0's l2: 3.48806e+12
[9300]	valid_0's l2: 3.48531e+12
[9400]	valid_0'

In [264]:
preds = []
for model in models :
    preds.append(model.predict(X_test))
len(preds)

10

In [265]:
import numpy as np
prediction = np.mean(preds, axis=0)

In [266]:
prediction

array([691695.2517164 , 672319.21502713, 851316.339441  , 903897.40003323,
       903897.40003323, 635577.41282796, 626110.17356157, 625556.93486383,
       625560.00726256, 691239.28514796, 635820.24005134, 691695.2517164 ,
       691695.2517164 , 625676.61787807, 851316.339441  , 955633.49804001,
       625999.87401811, 851316.339441  , 672319.21502713, 757723.25541921,
       625556.93486383, 627289.84345835, 635820.24005134, 625556.93486383,
       625556.93486383, 955633.49804001, 625808.53751061, 903897.40003323,
       625999.8043815 , 625556.93486383, 851316.339441  , 627269.50418685,
       626109.65018459, 691239.28514796, 626110.17356157, 627251.25526062,
       625676.54824146, 626109.65018459, 625999.8043815 , 625556.93486383,
       625556.93486383, 627251.25526062, 757723.25541921, 630908.00879904,
       757723.25541921, 851316.339441  , 625556.93486383, 635577.41282796,
       757723.25541921, 625560.00726256, 627269.50418685, 757723.25541921,
       625999.8043815 , 7

## sklearn_RandomForestRegressor 
+ RMSE + Hyper Parameter Tuning

* cv = 20 : 1460717.170338258	
* cv = 35 : 1454429.7489887767	
* 전처리 변경 후  : 1485610.1551896033
* cv = 19 : ?

In [176]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=300,
                              n_jobs=-1,
                              random_state=37)

X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']

In [177]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 300

# 랜덤 서치 횟수
num_epoch = 100

# hyperparameter 탐색 결과가 저장될 리스트
coarse_hyperparameters_list = []

for epoch in range(num_epoch):
    # max depth의 값을 2~100으로 설정
    max_depth = np.random.randint(low=2, high=100)

    # 적용할 feature의 비율을 0.1~1.0로 설정
    max_features = np.random.uniform(low=0.1, high=1.0)

    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=37)

    # cross_val_score를 실행하여 rmse의 평균 계산
    # cv는 데이터 조각 개수
    score = cross_val_score(model, X_train, Y_train, cv=35, scoring=rmse_score).mean() 
    
    # hyperparameter 탐색 결과를 저장할 딕셔너리
    hyperparameters = {
        'epoch': epoch,
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
        'cv' : cv_turn
        
    }

    # hyperparameter 탐색 결과 저장
    coarse_hyperparameters_list.append(hyperparameters)

    # hyperparameter 탐색 결과를 출력
    print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}, CV = {cv_turn}")

# coarse_hyperparameters_list를 DataFrame으로 변환
coarse_hyperparameters_list = pd.DataFrame.from_dict(coarse_hyperparameters_list)

# coarse_hyperparameters_list를 score가 낮은 순으로 정렬
coarse_hyperparameters_list = coarse_hyperparameters_list.sort_values(by="score")

print(coarse_hyperparameters_list.shape)

# coarse_hyperparameters_list의 상위 10개 출력
coarse_hyperparameters_list.head(10)

 0 n_estimators = 300, max_depth = 60, max_features = 0.616707, Score = 1315496.35414, CV = 35
 1 n_estimators = 300, max_depth = 79, max_features = 0.991063, Score = 1357078.74720, CV = 35
 2 n_estimators = 300, max_depth = 60, max_features = 0.323687, Score = 1259641.38147, CV = 35
 3 n_estimators = 300, max_depth = 38, max_features = 0.176827, Score = 1230452.67923, CV = 35
 4 n_estimators = 300, max_depth = 99, max_features = 0.237721, Score = 1238381.40219, CV = 35
 5 n_estimators = 300, max_depth = 75, max_features = 0.277449, Score = 1259641.38147, CV = 35
 6 n_estimators = 300, max_depth =  9, max_features = 0.141158, Score = 1229060.07320, CV = 35
 7 n_estimators = 300, max_depth = 30, max_features = 0.765452, Score = 1337917.07304, CV = 35
 8 n_estimators = 300, max_depth = 98, max_features = 0.678688, Score = 1325076.44314, CV = 35
 9 n_estimators = 300, max_depth = 15, max_features = 0.315257, Score = 1262136.58823, CV = 35
10 n_estimators = 300, max_depth = 60, max_feature

87 n_estimators = 300, max_depth = 28, max_features = 0.197585, Score = 1238381.40219, CV = 35
88 n_estimators = 300, max_depth = 57, max_features = 0.142109, Score = 1230452.67923, CV = 35
89 n_estimators = 300, max_depth = 13, max_features = 0.801065, Score = 1344450.54718, CV = 35
90 n_estimators = 300, max_depth =  9, max_features = 0.990720, Score = 1357013.02241, CV = 35
91 n_estimators = 300, max_depth = 36, max_features = 0.954396, Score = 1357078.74720, CV = 35
92 n_estimators = 300, max_depth = 21, max_features = 0.709734, Score = 1329223.95579, CV = 35
93 n_estimators = 300, max_depth = 99, max_features = 0.890463, Score = 1368766.52920, CV = 35
94 n_estimators = 300, max_depth = 26, max_features = 0.679421, Score = 1324613.95429, CV = 35
95 n_estimators = 300, max_depth = 33, max_features = 0.766637, Score = 1337917.07304, CV = 35
96 n_estimators = 300, max_depth = 14, max_features = 0.957264, Score = 1367282.00229, CV = 35
97 n_estimators = 300, max_depth =  6, max_feature

,epoch,score,n_estimators,max_depth,max_features,cv
19,19,1228273.97141,300,10,0.11324,35
6,6,1229060.07320,300,9,0.14116,35
3,3,1230452.67923,300,38,0.17683,35
48,48,1230452.67923,300,32,0.17129,35
88,88,1230452.67923,300,57,0.14211,35
38,38,1230452.67923,300,27,0.11099,35
34,34,1230452.67923,300,92,0.12902,35
99,99,1232328.97691,300,5,0.19866,35
82,82,1234135.08445,300,19,0.15126,35
58,58,1234828.39771,300,5,0.12831,35


In [178]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 300
num_epoch = 100
finer_hyperparameters_list = []

for epoch in range(num_epoch):
    # coarse 단계에서 찾은 뎁스와 피쳐비율 구간을 적용
    max_depth = np.random.randint(low=5, high=35)
    max_features = np.random.uniform(low=0.1, high=0.3)
    
    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=37)

    score = cross_val_score(model, X_train, Y_train, cv=35, scoring=rmse_score).mean()
    hyperparameters = {
        'epoch': epoch,
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
    }

    finer_hyperparameters_list.append(hyperparameters)
    print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}, CV = {cv_turn}")

finer_hyperparameters_list = pd.DataFrame.from_dict(finer_hyperparameters_list)
finer_hyperparameters_list = finer_hyperparameters_list.sort_values(by="score")
print(finer_hyperparameters_list.shape)

finer_hyperparameters_list.head(10)

 0 n_estimators = 300, max_depth = 24, max_features = 0.154987, Score = 1230442.97181, CV = 35
 1 n_estimators = 300, max_depth = 23, max_features = 0.200939, Score = 1238911.18673, CV = 35
 2 n_estimators = 300, max_depth = 30, max_features = 0.281927, Score = 1259641.38147, CV = 35
 3 n_estimators = 300, max_depth = 27, max_features = 0.177031, Score = 1230452.67923, CV = 35
 4 n_estimators = 300, max_depth = 21, max_features = 0.103141, Score = 1229122.15659, CV = 35
 5 n_estimators = 300, max_depth = 27, max_features = 0.124725, Score = 1230452.67923, CV = 35
 6 n_estimators = 300, max_depth = 15, max_features = 0.225759, Score = 1233716.83442, CV = 35
 7 n_estimators = 300, max_depth = 28, max_features = 0.214460, Score = 1238381.40219, CV = 35
 8 n_estimators = 300, max_depth = 15, max_features = 0.270541, Score = 1233716.83442, CV = 35
 9 n_estimators = 300, max_depth = 11, max_features = 0.161569, Score = 1228560.08702, CV = 35
10 n_estimators = 300, max_depth = 17, max_feature

87 n_estimators = 300, max_depth = 10, max_features = 0.266491, Score = 1237187.85290, CV = 35
88 n_estimators = 300, max_depth = 31, max_features = 0.235387, Score = 1238381.40219, CV = 35
89 n_estimators = 300, max_depth = 34, max_features = 0.132489, Score = 1230452.67923, CV = 35
90 n_estimators = 300, max_depth = 31, max_features = 0.239327, Score = 1238381.40219, CV = 35
91 n_estimators = 300, max_depth =  6, max_features = 0.230084, Score = 1240480.91709, CV = 35
92 n_estimators = 300, max_depth = 30, max_features = 0.134809, Score = 1230452.67923, CV = 35
93 n_estimators = 300, max_depth = 10, max_features = 0.219042, Score = 1237187.85290, CV = 35
94 n_estimators = 300, max_depth = 29, max_features = 0.122125, Score = 1230452.67923, CV = 35
95 n_estimators = 300, max_depth = 20, max_features = 0.146477, Score = 1230244.63515, CV = 35
96 n_estimators = 300, max_depth = 24, max_features = 0.135624, Score = 1230442.97181, CV = 35
97 n_estimators = 300, max_depth = 21, max_feature

,epoch,score,n_estimators,max_depth,max_features
21,21,1220254.70002,300,14,0.16057
20,20,1228560.08702,300,11,0.17771
9,9,1228560.08702,300,11,0.16157
59,59,1229060.07320,300,9,0.15754
4,4,1229122.15659,300,21,0.10314
66,66,1229131.41108,300,22,0.10230
13,13,1229131.41108,300,22,0.11379
39,39,1229667.30714,300,23,0.17927
95,95,1230244.63515,300,20,0.14648
37,37,1230407.57142,300,26,0.13197


In [179]:
# 가장 정확도가 높은 하이퍼패러미터를 저장
best_hyperparameters = finer_hyperparameters_list.iloc[0]

best_max_depth = best_hyperparameters["max_depth"]
best_max_features = best_hyperparameters["max_features"]

print(f"max_depth(best) = {best_max_depth}, max_features(best) = {best_max_features:.6f}")

max_depth(best) = 14.0, max_features(best) = 0.160568


In [180]:
from sklearn.ensemble import RandomForestRegressor

best_n_estimators = 3000
model = RandomForestRegressor(n_estimators=best_n_estimators,
                              max_depth=best_max_depth,
                              max_features=best_max_features,
                              random_state=37,
                              n_jobs=-1)
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14.0, max_features=0.1605682313332864,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                      warm_start=False)

In [181]:
model.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14.0, max_features=0.1605682313332864,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                      warm_start=False)

In [182]:
prediction = model.predict(X_test)

In [183]:
prediction

array([1.15038092e+06, 1.24903780e+06, 1.75701596e+06, 2.07221512e+06,
       1.39394526e+06, 9.85513121e+04, 2.17641340e+04, 2.79651935e+04,
       4.62429072e+04, 3.58212689e+05, 4.93367853e+05, 8.94959744e+05,
       4.87719941e+05, 4.82640362e+04, 1.49368497e+06, 2.12248821e+06,
       1.41599101e+05, 7.14404755e+05, 9.22472270e+05, 4.21125906e+05,
       8.00428033e+04, 1.30441565e+05, 2.83072923e+05, 2.41176227e+05,
       5.21104728e+04, 1.96295559e+06, 4.84846508e+04, 1.70408162e+06,
       2.26941826e+04, 8.20375452e+04, 1.40711813e+06, 2.28250470e+05,
       4.16201926e+04, 7.25130927e+05, 3.07799634e+04, 4.41065559e+04,
       4.37074962e+05, 7.45929566e+03, 3.86512878e+04, 1.36833906e+04,
       4.36192402e+04, 2.63279452e+05, 9.84984294e+05, 3.73246498e+05,
       1.07770354e+06, 1.43494929e+06, 1.62516690e+05, 5.40836088e+04,
       3.68597944e+05, 1.08846824e+04, 1.68690576e+04, 7.47186978e+05,
       6.12990060e+04, 1.50043341e+06, 1.41899408e+06, 8.86838762e+04,
      

### 최적의 cv 찾기
* 35로 사용 : 1454429.7489887767	
* 추가적으로 더 해볼 수 있으면 ? 해봐도 될듯
* overfitting 일어나는 것 같아.........ㅠ

In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=37)

X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 100

# 랜덤 서치 횟수
num_epoch = 100

In [13]:
cv_cv = []
for cv_turn in range(27, 36, 2) : # 5, 7, 9, 11, 13, 15, 17, 19


# hyperparameter 탐색 결과가 저장될 리스트
    coarse_hyperparameters_list = []

    for epoch in range(num_epoch):
        # max depth의 값을 2~100으로 설정
        max_depth = np.random.randint(low=2, high=10)

        # 적용할 feature의 비율을 0.1~1.0로 설정
        max_features = np.random.uniform(low=0.1, high=1.0)

        model = RandomForestRegressor(n_estimators=n_estimators,
                                      max_depth=max_depth,
                                      max_features=max_features,
                                      n_jobs=-1,
                                      random_state=37)

        # cross_val_score를 실행하여 rmse의 평균 계산
        # cv는 데이터 조각 개수
        score = cross_val_score(model, X_train, Y_train, cv=cv_turn, scoring=rmse_score).mean() 

        # hyperparameter 탐색 결과를 저장할 딕셔너리
        hyperparameters = {
            'epoch': epoch,
            'score': score,
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'max_features': max_features,
            'cv' : cv_turn
        }

        # hyperparameter 탐색 결과 저장
        coarse_hyperparameters_list.append(hyperparameters)

        # hyperparameter 탐색 결과를 출력
        print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}, CV = {cv_turn}")

    # coarse_hyperparameters_list를 DataFrame으로 변환
    coarse_hyperparameters_list = pd.DataFrame.from_dict(coarse_hyperparameters_list)

    # coarse_hyperparameters_list를 score가 낮은 순으로 정렬
    coarse_hyperparameters_list = coarse_hyperparameters_list.sort_values(by="score")

    # coarse_hyperparameters_list의 상위 1개 저장
    cv_cv.append(coarse_hyperparameters_list.head(1))

 0 n_estimators = 100, max_depth =  5, max_features = 0.509878, Score = 1325695.48753, CV = 27
 1 n_estimators = 100, max_depth =  8, max_features = 0.127115, Score = 1281109.04509, CV = 27
 2 n_estimators = 100, max_depth =  7, max_features = 0.803308, Score = 1366464.20371, CV = 27
 3 n_estimators = 100, max_depth =  3, max_features = 0.754579, Score = 1309221.80309, CV = 27
 4 n_estimators = 100, max_depth =  4, max_features = 0.544358, Score = 1302641.26623, CV = 27
 5 n_estimators = 100, max_depth =  4, max_features = 0.369654, Score = 1302376.31148, CV = 27
 6 n_estimators = 100, max_depth =  2, max_features = 0.488633, Score = 1296731.61885, CV = 27
 7 n_estimators = 100, max_depth =  4, max_features = 0.983320, Score = 1323776.82688, CV = 27
 8 n_estimators = 100, max_depth =  6, max_features = 0.630222, Score = 1349240.69718, CV = 27
 9 n_estimators = 100, max_depth =  4, max_features = 0.498273, Score = 1302641.26623, CV = 27
10 n_estimators = 100, max_depth =  4, max_feature

87 n_estimators = 100, max_depth =  9, max_features = 0.521090, Score = 1370102.21906, CV = 27
88 n_estimators = 100, max_depth =  6, max_features = 0.963370, Score = 1361834.88721, CV = 27
89 n_estimators = 100, max_depth =  8, max_features = 0.215690, Score = 1311723.68964, CV = 27
90 n_estimators = 100, max_depth =  8, max_features = 0.206673, Score = 1311723.68964, CV = 27
91 n_estimators = 100, max_depth =  6, max_features = 0.723667, Score = 1358743.19192, CV = 27
92 n_estimators = 100, max_depth =  6, max_features = 0.381862, Score = 1307298.46729, CV = 27
93 n_estimators = 100, max_depth =  6, max_features = 0.438002, Score = 1307298.46729, CV = 27
94 n_estimators = 100, max_depth =  9, max_features = 0.246318, Score = 1293461.15986, CV = 27
95 n_estimators = 100, max_depth =  6, max_features = 0.530163, Score = 1333298.11534, CV = 27
96 n_estimators = 100, max_depth =  3, max_features = 0.912272, Score = 1320606.08328, CV = 27
97 n_estimators = 100, max_depth =  9, max_feature

74 n_estimators = 100, max_depth =  5, max_features = 0.404324, Score = 1273304.73295, CV = 29
75 n_estimators = 100, max_depth =  2, max_features = 0.996799, Score = 1275429.37831, CV = 29
76 n_estimators = 100, max_depth =  9, max_features = 0.407121, Score = 1333030.11289, CV = 29
77 n_estimators = 100, max_depth =  8, max_features = 0.232042, Score = 1253951.14701, CV = 29
78 n_estimators = 100, max_depth =  2, max_features = 0.926369, Score = 1275429.37831, CV = 29
79 n_estimators = 100, max_depth =  6, max_features = 0.152489, Score = 1265572.22308, CV = 29
80 n_estimators = 100, max_depth =  9, max_features = 0.877073, Score = 1346614.88957, CV = 29
81 n_estimators = 100, max_depth =  6, max_features = 0.233670, Score = 1255456.23439, CV = 29
82 n_estimators = 100, max_depth =  8, max_features = 0.916982, Score = 1334796.29407, CV = 29
83 n_estimators = 100, max_depth =  8, max_features = 0.395373, Score = 1300676.48351, CV = 29
84 n_estimators = 100, max_depth =  9, max_feature

61 n_estimators = 100, max_depth =  9, max_features = 0.328982, Score = 1305329.26460, CV = 31
62 n_estimators = 100, max_depth =  6, max_features = 0.832604, Score = 1314073.71769, CV = 31
63 n_estimators = 100, max_depth =  2, max_features = 0.160584, Score = 1376707.81149, CV = 31
64 n_estimators = 100, max_depth =  5, max_features = 0.213008, Score = 1271857.50336, CV = 31
65 n_estimators = 100, max_depth =  9, max_features = 0.445805, Score = 1336109.38798, CV = 31
66 n_estimators = 100, max_depth =  4, max_features = 0.883636, Score = 1296350.98008, CV = 31
67 n_estimators = 100, max_depth =  6, max_features = 0.484503, Score = 1305406.05898, CV = 31
68 n_estimators = 100, max_depth =  6, max_features = 0.524694, Score = 1305406.05898, CV = 31
69 n_estimators = 100, max_depth =  5, max_features = 0.554337, Score = 1281085.21173, CV = 31
70 n_estimators = 100, max_depth =  2, max_features = 0.120180, Score = 1376707.81149, CV = 31
71 n_estimators = 100, max_depth =  8, max_feature

48 n_estimators = 100, max_depth =  5, max_features = 0.108224, Score = 1250081.28961, CV = 33
49 n_estimators = 100, max_depth =  9, max_features = 0.361902, Score = 1299165.12864, CV = 33
50 n_estimators = 100, max_depth =  4, max_features = 0.186818, Score = 1244136.16291, CV = 33
51 n_estimators = 100, max_depth =  5, max_features = 0.529432, Score = 1295755.60739, CV = 33
52 n_estimators = 100, max_depth =  5, max_features = 0.733720, Score = 1299846.09778, CV = 33
53 n_estimators = 100, max_depth =  7, max_features = 0.263430, Score = 1256199.00852, CV = 33
54 n_estimators = 100, max_depth =  7, max_features = 0.386467, Score = 1305772.04029, CV = 33
55 n_estimators = 100, max_depth =  9, max_features = 0.535032, Score = 1323601.30685, CV = 33
56 n_estimators = 100, max_depth =  4, max_features = 0.165351, Score = 1252993.08216, CV = 33
57 n_estimators = 100, max_depth =  6, max_features = 0.803342, Score = 1323252.57793, CV = 33
58 n_estimators = 100, max_depth =  6, max_feature

35 n_estimators = 100, max_depth =  6, max_features = 0.301754, Score = 1279762.31221, CV = 35
36 n_estimators = 100, max_depth =  4, max_features = 0.886479, Score = 1287447.99745, CV = 35
37 n_estimators = 100, max_depth =  4, max_features = 0.272271, Score = 1244291.83599, CV = 35
38 n_estimators = 100, max_depth =  3, max_features = 0.327067, Score = 1248330.84100, CV = 35
39 n_estimators = 100, max_depth =  8, max_features = 0.492819, Score = 1309643.76943, CV = 35
40 n_estimators = 100, max_depth =  5, max_features = 0.309829, Score = 1251316.56534, CV = 35
41 n_estimators = 100, max_depth =  2, max_features = 0.636533, Score = 1260845.37865, CV = 35
42 n_estimators = 100, max_depth =  2, max_features = 0.352842, Score = 1263268.70901, CV = 35
43 n_estimators = 100, max_depth =  5, max_features = 0.356045, Score = 1251316.56534, CV = 35
44 n_estimators = 100, max_depth =  7, max_features = 0.250349, Score = 1253568.87927, CV = 35
45 n_estimators = 100, max_depth =  3, max_feature

In [14]:
display(cv_cv)

[    epoch         score  n_estimators  max_depth  max_features  cv
 60     60  1.267566e+06           100          5      0.221966  27,
     epoch         score  n_estimators  max_depth  max_features  cv
 54     54  1.246747e+06           100          7        0.1176  29,
     epoch         score  n_estimators  max_depth  max_features  cv
 17     17  1.259830e+06           100          9      0.142936  31,
     epoch         score  n_estimators  max_depth  max_features  cv
 88     88  1.235076e+06           100          6      0.233692  33,
     epoch         score  n_estimators  max_depth  max_features  cv
 67     67  1.229384e+06           100          6      0.170716  35]

In [35]:
display(cv_cv)

[    epoch         score  n_estimators  max_depth  max_features  cv
 25     25  1.303383e+06           100          5      0.234854  19,
     epoch         score  n_estimators  max_depth  max_features  cv
 56     56  1.277253e+06           100          8      0.174468  21,
     epoch         score  n_estimators  max_depth  max_features  cv
 12     12  1.328894e+06           100          7      0.178809  23,
     epoch         score  n_estimators  max_depth  max_features  cv
 60     60  1.266826e+06           100          9      0.110251  25]

In [ ]:
# 가장 정확도가 높은 하이퍼패러미터를 저장
best_hyperparameters = finer_hyperparameters_list.iloc[0]

best_max_depth = best_hyperparameters["max_depth"]
best_max_features = best_hyperparameters["max_features"]

print(f"max_depth(best) = {best_max_depth}, max_features(best) = {best_max_features:.6f}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

best_n_estimators = 3000
model = RandomForestRegressor(n_estimators=best_n_estimators,
                              max_depth=best_max_depth,
                              max_features=best_max_features,
                              random_state=37,
                              n_jobs=-1)
model

In [ ]:
model.fit(X_train, Y_train)
prediction = model.predict(X_test)

## tensorflow

In [16]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [24]:
import tensorflow as tf
import numpy as np

X = tf.placeholder(shape = [None,11], dtype=tf.float32)
Y = tf.placeholder(shape = [None,1], dtype=tf.float32)

W = tf.Variable(tf.random_normal([11,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

H = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(H-Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0000001)
train = optimizer.minimize(cost)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(5000) :
    _, cost_val = sess.run([train, cost],
                          feed_dict = {
                              X : train_X.values,
                              Y : train_Y.values.reshape(-1,1)
                          })
    if step % 500 == 0 :
        print('cost값은 : {}'.format(cost_val))
    
prediction = sess.run(H, feed_dict = {X : test_X.values, })

cost값은 : 3833388597248.0
cost값은 : 2351955443712.0
cost값은 : 2347802558464.0
cost값은 : 2347764809728.0
cost값은 : 2347744100352.0
cost값은 : 2347723653120.0
cost값은 : 2347703730176.0
cost값은 : 2347683807232.0
cost값은 : 2347663622144.0
cost값은 : 2347643174912.0


# 저장

In [27]:
# 저장함수
def save_file(prediction, number) :
    sub = pd.read_csv('./data/submission.csv')
    sub['box_off_num'] = prediction
    sub.to_csv('./submission/submission' + str(number) + '.csv', index=False)
    return print('done')

In [184]:
save_file(prediction, 111)

done


In [160]:
a = 0
for i in range(len(prediction)) :
    if prediction[i] <= 0 :
        prediction[i] = 0
        a += 1

print(a)
#prediction = prediction.astype(int)
prediction

71


array([1115566.30565657, 1330898.07271574, 1956710.48447819,
       2166859.56483689, 2080734.17510612,  346045.45931293,
             0.        ,       0.        ,  112417.19426519,
        951379.95868599, 1162406.66919744, 1679870.66858181,
       1124792.64272501,   49453.00864089, 1559751.63842055,
       2517328.07425241,       0.        , 1568512.0582598 ,
       1354633.42175171,  243400.04957435,       0.        ,
        450034.0288779 ,  454500.39016113,       0.        ,
             0.        , 2425606.73876284,       0.        ,
       1806573.56296934,   87379.25725834,  204802.27780387,
       1488231.30625899,  218700.55519663,       0.        ,
       1374517.85637328,   86868.77794634,       0.        ,
             0.        ,       0.        ,   22318.97752276,
             0.        ,  204638.03387941,  397508.22867611,
        374282.20315428,  543604.79166777,  867479.27053258,
       2054273.09964444,       0.        ,  273394.65591729,
        314385.90759814,